## 단어를 인덱스로 변경하기
- 출처는 핸즈 온 머신러닝 책임.

- 우선 샘플 데이터를 가지고 단어->인덱스로 일종의 사전을 만든다. `tf.lookup.KeyValueTensorInitializer`
```
>> vocab = ["pathfinder", "arc", "hero", "adelle"]
>> indices = tf.range(len(vocab), dtype=tf.int64) # [0,1,2,3,4]
>> table_init=tf.lookup.KeyValueTensorInitializer(vocab, indices)
```
- 테이블을 만들어주어야 lookup이 가능하다. 단, 사전에 없는 단어가 들어올 경우 만들어 줄 새로운 인덱스의 크기를 num_oov_buckets으로 지정. `tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)`
```
>> num_oov_buckets = 2
>> table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)
```
- 내가 변환하고 싶은 리스트를 테이블에서 lookup을 해주면, 단어가 인덱스로 변환이 되어 있다. 
```
>> examples=tf.constant(["adelle", "hoyoung", "arc", "arc" , "bowmaster", "pathfinder"])
>> examples_indices=table.lookup(examples)
<tf.Tensor: shape=(6,), dtype=int64, numpy=array([3, 6, 1, 1, 6, 0], dtype=int64)>
```
- 원 핫 인코딩으로 변환할때 depth는 사전의 크기만큼.
```
>> one_hot = tf.one_hot(examples_indices, depth=len(vocab)+num_oov_buckets)
 array([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0., 0.]], dtype=float32)
```

## 인덱스 -> 임베딩 만드는 방법

- 우선 인덱스로 구성된 데이터셋을 준비한다. 
- `model.add(Embedding(사전크기, 피쳐별차원수,피쳐수))`
- `피쳐수=maxlen=자연어 처리 할 때 문장의 기본 길이=train_x.shape[1]`로 이해하면 될 듯

```
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

keras.backend.clear_session()  
np.random.seed(40)
tf.random.set_seed(40)

model = Sequential()

model.add(Embedding(50, 10, input_length=train_x.shape[1]))
model.add(Flatten())
model.add(Dense(16, activation='relu', input_shape=(10,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

model.fit(train_x, train_y, epochs=50, batch_size=4, verbose=1)
```

## 인덱스 -> 원핫 -> 임베딩
- 사전을 제작함  
```
vocab=["male","female"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init=tf.lookup.KeyValueTensorInitializer(vocab, indices)
table = tf.lookup.StaticVocabularyTable(table_init, 2)
```  
- 사전에서 룩업해서 인덱스로 변환함  
```
examples=tf.constant(train['Sex'])
examples_indices=table.lookup(examples)
```  
- 원핫 인코딩함  
```
one_hot = tf.one_hot(examples_indices, depth=len(vocab)+2)
```  
- 모델 학습   

```  
one_hot = tf.one_hot(examples_indices, depth=len(vocab)+2)

x_train = one_hot
y_train = np.ravel(train.Survived)

keras.backend.clear_session()  
np.random.seed(40)
tf.random.set_seed(40)

model = Sequential()

model.add(Embedding(50, 10, input_length=x_train.shape[1]))
model.add(Flatten())
model.add(Dense(16, activation='relu', input_shape=(10,)))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size=4, verbose=1)
```  

## 여러가지 피쳐 -> 인덱스 -> 원핫 -> concat -> 임베딩
- `tf.concat([t1,t2],axis=1)` 오른쪽으로 concat
- `tf.concat([t1,t2],axis=0)` 아래쪽으로 concat
- 따라서 train 데이터 생성시에 `x_train = tf.concat([one_hot,one_hot2,one_hot],axis=1)`으로 여러 원핫 인코딩을 concat해주면 된다. 